<a href="https://colab.research.google.com/github/nataliawcislo/jobposting_dataset_analysis/blob/main/clened_dataser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  print(len(fn))

Saving DataScientist 2.csv to DataScientist 2 (1).csv
User uploaded file "DataScientist 2.csv" with length 14957616 bytes
19


In [ ]:
# Importing libraries for analysis in alphabetical order
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go

from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# show matplotlib charts in cells
%matplotlib inline


# Set display options to not include scientific notation 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
data_s = pd.read_csv(fn)
data_s.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.50,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.50,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.00,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.40,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.40,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


In [ ]:
pd.set_option("max_rows", None)
data_s

In [ ]:
# cleaning  position name in job title
data_s['Job Title'] = data_s['Job Title'].str.replace('Sr.', 'Senior')
data_s['Job Title'] = data_s['Job Title'].str.replace('SR', 'Senior')
data_s['Job Title'] = data_s['Job Title'].str.replace('SR.', 'Senior')
data_s['Job Title'] = data_s['Job Title'].str.replace('Sr.', 'Senior')
data_s['Job Title'] = data_s['Job Title'].str.replace('Jr.', 'Junior')
data_s['Job Title'] = data_s['Job Title'].str.replace('Jr', 'Junior')
data_s['Job Title'] = data_s['Job Title'].str.replace('JR', 'Junior')
data_s['Job Title'] = data_s['Job Title'].str.replace('Jr.', 'Junior')
data_s['Job Title'] = data_s['Job Title'].str.replace('internship', 'Intern')
data_s['Job Title'] = data_s['Job Title'].str.replace('Internship', 'Intern')
data_s['Job Title'] = data_s['Job Title'].str.replace('Intern', 'Intern')
data_s['Job Title'] = data_s['Job Title'].str.replace('SeniorData Analyst', 'Senior Data Analyst')
data_s['Job Title'] = data_s['Job Title'].str.replace('SeniorData Scientist', 'Senior Data Scientist')
data_s['Job Title'] = data_s['Job Title'].str.replace('12 month Roster', ' ')
data_s['Job Title'] = data_s['Job Title'].str.replace('Direct Hire', ' ')

In [ ]:
data_s = data_s.drop(data_s.index[data_s['Founded']== -1])
data_s = data_s.drop(data_s.index[data_s['Rating']== -1.00])
data_s = data_s.drop(data_s.index[data_s['Type of ownership']== 'Unknown / Non-Applicable'])
data_s = data_s.drop(data_s.index[data_s['Job Title']== 'Medical Laboratory'])
data_s = data_s.drop(data_s.index[data_s['Job Title']== 'Lab'])
data_s = data_s.drop(data_s.index[data_s['Job Description']== 'chemistry'])

In [ ]:
# categorization for company sizes
small =  ["1 to 50 employees", "51 to 200 employees"]
medium = ["201 to 500 employees", '501 to 1000 employees']
large = ['10000+ employees', '1001 to 5000 employees', '5001 to 10000 employees']


#small
data_s["Size"] = data_s["Size"].replace(small, 1) 
#Medium
data_s["Size"] = data_s["Size"].replace(medium, 1)
#Large
data_s["Size"] = data_s["Size"].replace(large, 3)

In [ ]:
data_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2899 entries, 0 to 3906
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2899 non-null   int64  
 1   index              2899 non-null   int64  
 2   Job Title          2899 non-null   object 
 3   Salary Estimate    2899 non-null   object 
 4   Job Description    2899 non-null   object 
 5   Rating             2899 non-null   float64
 6   Company Name       2899 non-null   object 
 7   Location           2899 non-null   object 
 8   Headquarters       2899 non-null   object 
 9   Size               2899 non-null   object 
 10  Founded            2899 non-null   int64  
 11  Type of ownership  2899 non-null   object 
 12  Industry           2899 non-null   object 
 13  Sector             2899 non-null   object 
 14  Revenue            2899 non-null   object 
 15  Competitors        2899 non-null   object 
 16  Easy Apply         2899 

In [ ]:
#separate columns of Salary Estimate as minimum and maximum salary

df_salary = data_s['Salary Estimate'].str.split("-",expand=True,)

minimum_salary = df_salary[0]
minimum_salary = minimum_salary.str.replace('K','000')


maximum_salary = df_salary[1].str.replace('(Glassdoor est.)', ' ')
maximum_salary = maximum_salary.str.replace('(', ' ')
maximum_salary = maximum_salary.str.replace(')', ' ')
maximum_salary = maximum_salary.str.replace('K', '000')
maximum_salary = maximum_salary.str.replace('Employer est.', ' ')
maximum_salary = maximum_salary.str.replace('Per Hour', ' ')

maximum_salary = maximum_salary.str.replace('$', ' ').fillna(0).astype(int)
minimum_salary = minimum_salary.str.replace('$', ' ').fillna(0).astype(int)

data_s['Min'] = minimum_salary
data_s['Max'] = maximum_salary

In [ ]:
import re

HOURS_PER_WEEK = 40
WEEKS_PER_YEAR = 52
THOUSAND = 1000

def return_digits(x):
    result = re.findall(r'\d+', str(x))
    result = int(result[0]) if result else 0
    return result

def return_salary(string, isFrom):
    patternMain = None
    patternPerHour = None
    if(isFrom):
        patternMain = r'^\$\d+K';
        patternPerHour = r'^\$\d+';
    else:
        patternMain = r'-\$\d+K';
        patternPerHour = r'-\$\d+';
    
    result = None
    if('Per Hour' in string):
        result = re.findall(patternPerHour, str(string))
        result = return_digits(result[0]) if result else 0
        result = result * HOURS_PER_WEEK * WEEKS_PER_YEAR
    else:
        result = re.findall(patternMain, str(string))
        result = return_digits(result[0]) if result else 0
        result = result * THOUSAND
    return result

def return_average_salary(x):
    from_salary = return_salary(x, True)
    to_salary = return_salary(x, False)
    result = (from_salary+to_salary)/2
    return result

data_s['SalaryAverage'] =  data_s['Salary Estimate'].apply(return_average_salary)



data_s['avg'] = (data_s['Max'] + data_s['Min'])/2


In [ ]:
data_s['Founded'] = 2022-data_s['Founded']
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Company - Private', '3')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Private Practice / Firm', '3')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Contract', '3')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Other Organization	', '3')

data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Company - Public', '2')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Nonprofit Organization', '1')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('Government', '5')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('School / School District', '4')
data_s['Type of ownership'] = data_s['Type of ownership'].str.replace('College / University', '4')


data_s = data_s.drop(data_s.index[data_s['Type of ownership']== 'Hospital'])
data_s = data_s.drop(data_s.index[data_s['Type of ownership']== 'Subsidiary or Business Segment'])
data_s = data_s.drop(data_s.index[data_s['Type of ownership']== 'Unknown'])


In [ ]:
dataset = data_s[['Job Title', 'Job Description', 'avg', 'Founded','Size', 'Max', 'Min', 'Rating', 'Type of ownership' ]]

In [ ]:

dataset.head()

,Job Title,Job Description,avg,Founded,Size,Max,Min,Rating,Type of ownership
0,Senior Data Scientist,"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",146000.00,15,1,181000,111000,3.50,3
1,"Data Scientist, Product Analytics","At Noom, we use scientifically proven methods ...",146000.00,14,3,181000,111000,4.50,3
3,Data Analyst,Sapphire Digital seeks a dynamic and driven mi...,146000.00,3,1,181000,111000,3.40,3
4,"Director, Data Science","Director, Data Science - (200537)\nDescription...",146000.00,15,1,181000,111000,3.40,3
5,Data Scientist,Job Brief\n\nThe ideal candidate will have pre...,146000.00,37,1,181000,111000,2.90,3


In [ ]:
dataset['Job Title'] = data_s['Job Title'].str.replace('III.', '')
dataset['Job Title'] = data_s['Job Title'].str.replace('II.', '')
dataset['Job Title'] = data_s['Job Title'].str.replace('I.', '')
dataset['Job Title'] = data_s['Job Title'].str.replace('*', '')
dataset['Job Title'] = data_s['Job Title'].str.replace('CCB F&BM', '')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
import re
dataset['Level of expireince'] = data_s['Job Description'].str.findall('master|Master|ph.d|phd|msc|MSc|Bachelor|bachelor', flags=re.IGNORECASE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pd.set_option("max_rows", None)
dataset

In [ ]:
import pandas as pd
import re as re
def pick_only_key_sentence(str1, word):
    result = re.findall(r'([^.]*'+word+'[^.]*)', str1)
    return result

dataset['experience']=dataset['Job Description'].apply(lambda x : pick_only_key_sentence(x,'years of experience'))

print(dataset)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



                                              Job Title  ...                                         experience
0                                 Senior Data Scientist  ...                                                 []
1                     Data Scientist, Product Analytics  ...  [\nWhat We'll Like About You:\nYou have 4+ yea...
3                                          Data Analyst  ...  [) from four-year college or university; 4+ ye...
4                                Director, Data Science  ...                                                 []
5                                        Data Scientist  ...                                                 []
6                               Quantitative Researcher  ...                                                 []
8                                          AI Scientist  ...                                                 []
9                               Quantitative Researcher  ...                                            

In [ ]:
from google.colab import files
dataset.to_csv('out_n.csv', index=False)
files.download("out_n.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>